# PLY

To PLY είναι υλοποίηση σε Python για τα εργαλεία κατασκευής lex (λογισμικό παραγωγής λεκτικού αναλυτή) και yacc (λογισμικό παραγωγής συντακτικού αναλυτή)

## Δημιουργία απλού υπολογιστή πράξεων με το PLY

https://www.youtube.com/playlist?list=PLBOh8f9FoHHg7Ed_4yKhIbq4lIJAlonn8

In [24]:
import ply.lex as lex
import ply.yacc as yacc
import sys

In [25]:
# Δημιουργία tokens

tokens = [
    'INT', 
    'FLOAT',
    'NAME',
    'PLUS',
    'MINUS',
    'DIVIDE',
    'MULTIPLY',
    'EQUALS'
]

t_PLUS = r'\+'
t_MINUS = r'\-'
t_MULTIPLY = r'\*'
t_DIVIDE = r'\/'
t_EQUALS = r'\='

t_ignore = r' '

def t_FLOAT(t):
    r'\d+\.\d+'
    t.value = float(t.value)
    return t

def t_INT(t):
    r'\d+'
    t.value = int(t.value)
    return t

def t_NAME(t):
    r'[a-zA-Z_][a-zA-Z_0-9]*'
    t.type = 'NAME'
    return t

def t_error(t):
    print("Illegal characters!")
    t.lexer.skip(1)


In [26]:
# lexer

__file__ = "33-PLY.ipynb" # https://stackoverflow.com/questions/36393114/using-ply-with-ipython-jupyter

lexer = lex.lex()

lexer.input("x = 1 + 2")
while True:
    tok = lexer.token()
    if not tok:
        break
    print(tok)

LexToken(NAME,'x',1,0)
LexToken(EQUALS,'=',1,2)
LexToken(INT,1,1,4)
LexToken(PLUS,'+',1,6)
LexToken(INT,2,1,8)


In [27]:
# parser 

precedence = (
    ('left', 'PLUS', 'MINUS'),
    ('left', 'MULTIPLY', 'DIVIDE')
)

def p_calc(p):
    '''
    calc : expression
         | var_assign
         | empty
    '''
    print(p[1])

# expression
def p_expression(p):
    '''
    expression : expression MULTIPLY expression
               | expression DIVIDE expression
               | expression PLUS expression
               | expression MINUS expression
    '''
    p[0] = (p[2], p[1], p[3])

def p_expression_int_float(p):
    '''
    expression : INT
               | FLOAT
    '''
    p[0] = p[1]

def p_expression_var(p):
    '''
    expression : NAME
    '''
    p[0] = ('var', p[1])

# var_assign
def p_var_assign(p):
    '''
    var_assign : NAME EQUALS expression
    '''
    p[0] = ('=', p[1], p[3])

# empty
def p_empty(p):
    '''
    empty : 
    '''
    p[0] = None

def p_error(p):
    print("Syntax error")

parser = yacc.yacc(write_tables=False)

Generating LALR tables


In [28]:
s = 'x = 1 + a * b'
parser.parse(s)

('=', 'x', ('+', 1, ('*', ('var', 'a'), ('var', 'b'))))


In [29]:
s = ''
parser.parse(s)

None


In [47]:
# parser

precedence = (
    ('left', 'PLUS', 'MINUS'),
    ('left', 'MULTIPLY', 'DIVIDE')
)

def p_calc(p):
    '''
    calc : expression
         | var_assign
         | empty
    '''
    res = run(p[1])
    if res is not None:
        print(res)

# expression
def p_expression(p):
    '''
    expression : expression MULTIPLY expression
               | expression DIVIDE expression
               | expression PLUS expression
               | expression MINUS expression
    '''
    p[0] = (p[2], p[1], p[3])

def p_expression_int_float(p):
    '''
    expression : INT
               | FLOAT
    '''
    p[0] = p[1]

def p_expression_var(p):
    '''
    expression : NAME
    '''
    p[0] = ('var', p[1])

# var_assign
def p_var_assign(p):
    '''
    var_assign : NAME EQUALS expression
    '''
    p[0] = ('=', p[1], p[3])

# empty
def p_empty(p):
    '''
    empty : 
    '''
    p[0] = None

# error
def p_error(p):
    print("Syntax error")


locals={}
# run
def run(p):
    if type(p) == tuple:
        if p[0] == '+':
            return run(p[1]) + run(p[2])
        elif p[0] == '-':
            return run(p[1]) - run(p[2])
        elif p[0] == '*':
            return run(p[1]) * run(p[2])
        elif p[0] == '/':
            return run(p[1]) / run(p[2])
        elif p[0] == '=':
            locals[p[1]] = run(p[2])
            print(f"Variable '{p[1]}' set to {locals[p[1]]}")
        elif p[0] == 'var':
            if p[1] not in locals:
                return "Undeclared variable found!"
            else:
                return locals[p[1]]
    else:
        return p
    
parser = yacc.yacc(write_tables=False)

Generating LALR tables


In [48]:
s = '1 + 2 * 3'
parser.parse(s)

7


In [51]:
parser.parse('a = 1')
parser.parse('b = 99')
parser.parse('x = 7.5 * a + 8 * b')
parser.parse('x*2')

Variable 'a' set to 1
Variable 'b' set to 99
Variable 'x' set to 799.5
1599.0
